In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score, make_scorer

In [2]:
df_train = pd.read_csv('data/Yes_Bank_Train.csv')
df_orginal_test = pd.read_csv('data/Yes_Bank_Test_int.csv')

In [3]:
df_train.head()

,serial number,account_info,duration_month,credit_history,purpose,credit_amount,savings_account,employment_st,poi,personal_status,...,resident_since,property_type,age,installment_type,housing_type,credits_no,job_type,liables,telephone,foreigner
0,1,A11,6,A34,A43,1169,A65,A75,4,A93,...,4,A121,67,A143,A152,2,A173,1,A192,A201
1,2,A12,48,A32,A43,5951,A61,A73,2,A92,...,2,A121,22,A143,A152,1,A173,1,A191,A201
2,3,A14,12,A34,A46,2096,A61,A74,2,A93,...,3,A121,49,A143,A152,1,A172,2,A191,A201
3,4,A11,42,A32,A42,7882,A61,A74,2,A93,...,4,A122,45,A143,A153,1,A173,2,A191,A201
4,5,A11,24,A33,A40,4870,A61,A73,3,A93,...,4,A124,53,A143,A153,2,A173,2,A191,A201


In [4]:
df_orginal_test.head()

,serial number,account_info,duration_month,credit_history,purpose,savings_account,employment_st,poi,personal_status,gurantors,resident_since,property_type,age,installment_type,housing_type,credits_no,job_type,liables,telephone,foreigner
0,1,A14,24,A34,A46,A61,A75,4,A93,A101,4,A124,54,A143,A153,2,A173,2,A191,A201
1,2,A12,18,A34,A43,A61,A75,3,A92,A103,4,A121,48,A141,A151,2,A172,1,A192,A201
2,3,A11,20,A34,A42,A61,A75,1,A92,A101,4,A122,24,A143,A152,2,A173,1,A191,A201
3,4,A14,12,A34,A43,A65,A75,4,A93,A101,4,A123,35,A143,A152,2,A173,1,A191,A201
4,5,A12,12,A32,A40,A65,A71,1,A92,A101,2,A121,24,A143,A151,1,A171,1,A191,A201


In [5]:
df_train.drop(['serial number'], axis = 1, inplace=True)
df_test = df_orginal_test.drop(['serial number'], axis = 1)

In [6]:
df_train.shape

(800, 20)

In [7]:
df_test.shape

(200, 19)

In [8]:
df_train.isnull().sum()

account_info        0
duration_month      0
credit_history      0
purpose             0
credit_amount       0
savings_account     0
employment_st       0
poi                 0
personal_status     0
gurantors           0
resident_since      0
property_type       0
age                 0
installment_type    0
housing_type        0
credits_no          0
job_type            0
liables             0
telephone           0
foreigner           0
dtype: int64

In [9]:
# getting all the columns which are not integer and float, that is only strings
obj_df = df_train.select_dtypes(include=['object']).copy()
obj_df.head()

,account_info,credit_history,purpose,savings_account,employment_st,personal_status,gurantors,property_type,installment_type,housing_type,job_type,telephone,foreigner
0,A11,A34,A43,A65,A75,A93,A101,A121,A143,A152,A173,A192,A201
1,A12,A32,A43,A61,A73,A92,A101,A121,A143,A152,A173,A191,A201
2,A14,A34,A46,A61,A74,A93,A101,A121,A143,A152,A172,A191,A201
3,A11,A32,A42,A61,A74,A93,A103,A122,A143,A153,A173,A191,A201
4,A11,A33,A40,A61,A73,A93,A101,A124,A143,A153,A173,A191,A201


In [10]:
obj_df.shape

(800, 13)

In [11]:
object_columns = list(obj_df.columns)

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [13]:
# giving numerical values for each category for each column in the object-only dataframe
for column_name in object_columns:
    df_train["{}_code".format(column_name)] = le.fit_transform(obj_df[column_name])
    df_test["{}_code".format(column_name)] = le.transform(df_test[column_name])

In [14]:
df_train.drop(object_columns, axis = 1, inplace=True)
df_test.drop(object_columns, axis = 1, inplace=True)

In [15]:
df_train.head()

,duration_month,credit_amount,poi,resident_since,age,credits_no,liables,account_info_code,credit_history_code,purpose_code,savings_account_code,employment_st_code,personal_status_code,gurantors_code,property_type_code,installment_type_code,housing_type_code,job_type_code,telephone_code,foreigner_code
0,6,1169,4,4,67,2,1,0,4,4,4,4,2,0,0,2,1,2,1,0
1,48,5951,2,2,22,1,1,1,2,4,0,2,1,0,0,2,1,2,0,0
2,12,2096,2,3,49,1,2,3,4,7,0,3,2,0,0,2,1,1,0,0
3,42,7882,2,4,45,1,2,0,2,3,0,3,2,2,1,2,2,2,0,0
4,24,4870,3,4,53,2,2,0,3,0,0,2,2,0,3,2,2,2,0,0


In [23]:
X = df_train.drop(['credit_amount'], axis=1)
y = df_train['credit_amount']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [17]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor

from sklearn.model_selection import GridSearchCV

In [29]:
models = [
  [LinearRegression(), {"fit_intercept": [True, False]}], 
#   [SVR(), {"kernel": ["linear", "poly", "rbf", "sigmoid"]}], 
  [KNeighborsRegressor(), {"n_neighbors": [1,2,3,4,5,6,7], "weights": ["uniform", "distance"]}], 
  [DecisionTreeRegressor(), {"criterion": ["mse", "friedman_mse"], "splitter": ["best", "random"],
    "min_samples_split": [x for x in range(2,6)] # generates a list [2,3,4,5]
  }],
  [GradientBoostingRegressor(), {"loss": ["ls", "lad", "huber", "quantile"]}],
  [GaussianProcessRegressor(), {}],
#   [PLSRegression(), {}],
  [AdaBoostRegressor(), {}],
    [Lasso(), {"alpha": [0.2, 0.3, 0.7, 0.75, 0.8]}],
    [Ridge(), {"alpha": [0.2, 0.3, 0.7, 0.75, 0.8]}]
]

In [18]:
# the evaluation metric provided by competition rules
def evaluation_metric(y_test, y_pred):
    rmse = np.sqrt(np.mean((y_test-y_pred)**2))
    em = (1-rmse/100000)*100
    return em

In [30]:
# iteratively searching for right hyperparameters for the right model using 10 fold cross validation
# we measure both normalised rmse and r_sqaured metrics
for model in models:
    regressor = model[0]
    param_grid = model[1]
    model = GridSearchCV(regressor, param_grid, cv = 10)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(model.best_params_, model.best_estimator_)
    evaluation_metric_perf = evaluation_metric(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(evaluation_metric_perf, r2)

{'fit_intercept': False} LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)
98.08570031976174 0.5857829353768271
{'n_neighbors': 6, 'weights': 'uniform'} KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=6, p=2,
          weights='uniform')
97.78165101542587 0.44375275693434213
{'criterion': 'mse', 'min_samples_split': 3, 'splitter': 'random'} DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=3, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='random')
97.27718580153272 0.16199975358575525
{'loss': 'huber'} GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='huber', max_depth=3,
             max_features=None, max_leaf_nodes=No

In [ ]:
from sklearn.metrics.scorer import make_scorer
my_custom_scorer = make_scorer(r2_score, greater_is_better=True)

In [ ]:
# from tpot import TPOTRegressor

# tot_generations = [10, 25, 50, 100]
# # tot_generations = [1, 2, 3]

# for generation in tot_generations:
#     tpot = TPOTRegressor(generations=generation, population_size=100, verbosity=2, scoring = my_custom_scorer)
#     tpot.fit(X_train, y_train)
#     print(tpot.score(X_test, y_test))
#     tpot.export('teapee_yesbank__r2_gen_{}.py'.format(generation))

In [ ]:
# from tpot import TPOTRegressor

# # tot_generations = [10, 25, 50, 100]
# # tot_generations = [1, 2, 3]

# my_custom_em_scorer = make_scorer(evaluation_metric, greater_is_better=True)
# # for generation in tot_generations:
# tpot = TPOTRegressor(generations=10, population_size=100, verbosity=2, scoring = my_custom_em_scorer)
# tpot.fit(X_train, y_train)
# print(tpot.score(X_test, y_test))
# tpot.export('teapee_yesbank_em_gen_{}.py'.format(generation))

C:\Users\symphonyai\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


# xgboost

In [22]:
import xgboost as xgb
model = xgb.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.09,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
evaluation_metric_perf = evaluation_metric(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(evaluation_metric_perf, r2)

98.04784446949708 0.5692384172731351


# lgbm

In [19]:
import lightgbm as lgb

In [20]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [25]:
# specify your configurations as a dict
lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.2,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}


gbm = lgb.train(lgb_params,
                lgb_train,
                num_boost_round=100,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
evaluation_metric_perf = evaluation_metric(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(evaluation_metric_perf, r2)

[1]	valid_0's l1: 1698.01	valid_0's l2: 5.93445e+06
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 1562.87	valid_0's l2: 5.07878e+06
[3]	valid_0's l1: 1476.17	valid_0's l2: 4.53806e+06
[4]	valid_0's l1: 1419.74	valid_0's l2: 4.32357e+06
[5]	valid_0's l1: 1384.62	valid_0's l2: 4.06328e+06
[6]	valid_0's l1: 1366.3	valid_0's l2: 4.01627e+06
[7]	valid_0's l1: 1319	valid_0's l2: 3.84189e+06
[8]	valid_0's l1: 1320.6	valid_0's l2: 3.77055e+06
[9]	valid_0's l1: 1302.6	valid_0's l2: 3.65578e+06
[10]	valid_0's l1: 1309.13	valid_0's l2: 3.63494e+06
[11]	valid_0's l1: 1288.82	valid_0's l2: 3.54527e+06
[12]	valid_0's l1: 1278.22	valid_0's l2: 3.47494e+06
[13]	valid_0's l1: 1265.41	valid_0's l2: 3.46723e+06
[14]	valid_0's l1: 1260.53	valid_0's l2: 3.45865e+06
[15]	valid_0's l1: 1253.02	valid_0's l2: 3.46934e+06
[16]	valid_0's l1: 1237.15	valid_0's l2: 3.382e+06
[17]	valid_0's l1: 1218.05	valid_0's l2: 3.30886e+06
[18]	valid_0's l1: 1214.6	valid_0's l2: 3.30404e+06
[19

# catboost

In [27]:
from catboost import Pool, CatBoostRegressor

In [28]:
cat_params = {'depth': 11, 'iterations': 250, 'l2_leaf_reg': 9, 
        'learning_rate': 0.15, 'random_seed': 42,
        'loss_function': 'MAE'}
cat_features = [i for i in range(6,19)]
model = CatBoostRegressor(**cat_params)

train_data = Pool(X_train, y_train, cat_features=cat_features)
test_data = Pool(X_test, cat_features=cat_features)

model.fit(train_data, verbose=False)
y_pred = model.predict(test_data)

evaluation_metric_perf = evaluation_metric(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(evaluation_metric_perf, r2)

95.70881256792649 -1.0814361907118775


# ensemble

In [ ]:
# trying out multiple models for gaining a better score
estimator1 = GradientBoostingRegressor(loss='huber')
estimator2 = lgb_params
estimator3 = xgb.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.09,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)
estimator4 = CatBoostRegressor(**cat_params)

In [ ]:
estimator1.fit(X_train, y_train)
estimator2 = lgb.train(lgb_params,
                lgb_train,
                num_boost_round=25,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)
estimator3.fit(X_train, y_train)
estimator4.fit(train_data, verbose = False)

In [ ]:
y_pred1 = estimator1.predict(X_test)
y_pred2 = estimator2.predict(X_test, num_iteration=gbm.best_iteration)
y_pred3 = estimator3.predict(X_test)
y_pred4 = estimator4.predict(test_data)

In [27]:
def ensemble_four(y_test, y_pred):
    evaluation_metric_perf = evaluation_metric(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(evaluation_metric_perf, r2)

In [ ]:
ensemble_four(y_test, y_pred1)
ensemble_four(y_test, y_pred2)
ensemble_four(y_test, y_pred3)

In [ ]:
y_all_pred = np.mean(np.array([y_pred1, y_pred2, y_pred3]), axis = 0)

In [ ]:
ensemble_four(y_test, y_all_pred)

In [69]:
estimator1 = GradientBoostingRegressor(loss='huber')
estimator2 = lgb.train(lgb_params,
                lgb.Dataset(X_train, y_train),
                num_boost_round=21)
estimator3 = Lasso(alpha=0.9)

estimator1.fit(X_train, y_train)
# estimator2.fit(X_train, y_train)
estimator3.fit(X_train, y_train)

y_pred1 = estimator1.predict(X_test)
y_pred2 = estimator2.predict(X_test, num_iteration=gbm.best_iteration)
y_pred3 = estimator3.predict(X_test)

ensemble_four(y_test, y_pred1)
ensemble_four(y_test, y_pred2)
ensemble_four(y_test, y_pred3)

print('overall')
y_all_pred = np.mean(np.array([y_pred1, y_pred2, y_pred3]), axis = 0)
ensemble_four(y_test, y_all_pred)

98.20471472621848 0.5222224322981146
98.08015551549735 0.45362487474032454
98.18362044493321 0.5109288689826282
overall
98.21133863470696 0.5257415571653663


In [70]:
estimator1 = GradientBoostingRegressor(loss='huber')
estimator3 = Lasso(alpha=0.9)

estimator1.fit(X_train, y_train)
# estimator2.fit(X_train, y_train)
estimator3.fit(X_train, y_train)

y_pred1 = estimator1.predict(X_test)
# y_pred2 = estimator2.predict(X_test, num_iteration=gbm.best_iteration)
y_pred3 = estimator3.predict(X_test)

ensemble_four(y_test, y_pred1)
# ensemble_four(y_test, y_pred2)
ensemble_four(y_test, y_pred3)

print('overall')
y_all_pred = np.mean(np.array([y_pred1, y_pred3]), axis = 0)
ensemble_four(y_test, y_all_pred)

98.20570278234055 0.5227481884581526
98.18362044493321 0.5109288689826282
overall
98.23815974169429 0.539857998651009


In [48]:
estimator1 = GradientBoostingRegressor(loss='huber')
estimator2 = lgb_params


estimator1.fit(X_train, y_train)
estimator2 = lgb.train(lgb_params,
                lgb.Dataset(X_train, y_train),
                num_boost_round=21)


y_pred1 = estimator1.predict(X_test)
y_pred2 = estimator2.predict(X_test, num_iteration=gbm.best_iteration)

ensemble_four(y_test, y_pred1)
ensemble_four(y_test, y_pred2)

print('overall')

y_all_pred = np.mean(np.array([y_pred1, y_pred2]), axis = 0)
ensemble_four(y_test, y_all_pred)

98.2030009055437 0.5213098019545369
98.08015551549735 0.45362487474032454
overall
98.17553007210722 0.5065624030235485


# for actual test data

In [33]:
X_valid = df_test

In [34]:
X_valid.head()

,duration_month,poi,resident_since,age,credits_no,liables,account_info_code,credit_history_code,purpose_code,savings_account_code,employment_st_code,personal_status_code,gurantors_code,property_type_code,installment_type_code,housing_type_code,job_type_code,telephone_code,foreigner_code
0,24,4,4,54,2,2,3,4,7,0,4,2,0,3,2,2,2,0,0
1,18,3,4,48,2,1,1,4,4,0,4,1,2,0,0,0,1,1,0
2,20,1,4,24,2,1,0,4,3,0,4,1,0,1,2,1,2,0,0
3,12,4,4,35,2,1,3,4,4,4,4,2,0,2,2,1,2,0,0
4,12,1,2,24,1,1,1,2,0,4,0,1,0,0,2,0,0,0,0


In [78]:
estimator1 = GradientBoostingRegressor(loss='huber')
estimator2 = lgb_params


estimator1.fit(X, y)
estimator2 = lgb.train(lgb_params,
                lgb.Dataset(X, y),
                num_boost_round=21)


y_pred1 = estimator1.predict(X_valid)
y_pred2 = estimator2.predict(X_valid, num_iteration=gbm.best_iteration)

y_all_pred = np.mean(np.array([y_pred1, y_pred2]), axis = 0)

In [71]:
estimator1 = GradientBoostingRegressor(loss='huber')
estimator3 = Lasso(alpha=0.9)

estimator1.fit(X, y)
estimator3.fit(X, y)

y_pred1 = estimator1.predict(X_valid)
y_pred3 = estimator3.predict(X_valid)


y_all_pred = np.mean(np.array([y_pred1, y_pred3]), axis = 0)

In [72]:
len(y_all_pred)

200

In [79]:
y_all_pred

array([ 2515.90364746,  2245.93128828,  3514.13678618,  1471.42705135,
        4903.77316618,  7015.60348664,   813.29655609,  1204.63731769,
       10475.38240764,  1839.05192279,   892.35605686,   727.59060349,
        4901.03325828,  6211.15377451,  5669.64730395,  6029.42679495,
        1755.20900452,  2288.97739509,  7299.47534906,  1189.94033217,
        1185.86749943,  1300.11130121,  5000.75891756,   696.78388742,
        3057.37094228,  2197.67943123,  3695.36113921,  3115.52299563,
        4316.29960222,  6557.62576193,  2171.19633662,  1855.22600546,
        7333.3218511 ,  4053.16113231,  1615.15867923,  1444.541842  ,
        1283.57865997,  1747.924733  ,  2544.79567714,  1995.07961137,
        3559.64112033,  2800.98812492,  1119.64131492,  2450.79118828,
        2053.98137393,  4598.76746383,  3660.24138542,  1514.98454463,
        1716.76004223,   957.59921881,  1368.29954241,  3763.83649424,
        1749.46515469,  1634.10456222,  7131.413442  ,  1732.85273454,
      

In [74]:
df_submission = pd.DataFrame({'serial number': df_orginal_test['serial number'].values})

if rounded off values is required

In [75]:
df_submission['credit_amount'] = np.round(y_all_pred).astype('int64')

In [76]:
df_submission.head()

,serial number,credit_amount
0,1,2533
1,2,2440
2,3,3923
3,4,1354
4,5,4509


In [77]:
df_submission.to_csv('submission_ensembles_gbm_lasso_rounded.csv', index=None)

# classification thing

In [ ]:
# Cluster number 1 (Correct value is 1) : When the value of credit_amount is between 4000 and 20000 
# Cluster number 2 (Correct value is 2): When the value of credit_amount is between 4000 and 1500 
# Cluster number 3 (Correct value is 3) : When the value of credit_amount is less than 1500
# ["serial number", "cluster_number"] in df_submission

In [80]:
y_all_pred_cluster = list(y_all_pred)

In [81]:
cluster_result = []
for c_value in y_all_pred_cluster:
    if((c_value >= 4000) and (c_value <= 20000)):
        cluster_result.append(1)
    elif((c_value >= 1500) and (c_value < 4000)):
        cluster_result.append(2)
    elif c_value < 1500:
        cluster_result.append(3)

In [82]:
np.array(cluster_result)

array([2, 2, 2, 3, 1, 1, 3, 3, 1, 2, 3, 3, 1, 1, 1, 1, 2, 2, 1, 3, 3, 3,
       1, 3, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 2, 3, 3, 2, 2, 2, 2, 2, 3, 2,
       2, 1, 2, 2, 2, 3, 3, 2, 2, 2, 1, 2, 3, 2, 2, 2, 2, 2, 2, 1, 2, 2,
       2, 2, 1, 3, 1, 2, 2, 3, 2, 3, 1, 2, 1, 1, 1, 1, 2, 3, 2, 3, 2, 1,
       1, 1, 1, 3, 3, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 3, 2, 2, 2, 1, 2, 2,
       2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 3, 2, 2, 2, 2, 1, 2, 3, 2, 3,
       2, 3, 3, 1, 3, 3, 1, 1, 1, 2, 2, 2, 3, 1, 2, 2, 2, 2, 2, 1, 2, 1,
       3, 2, 2, 2, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2, 1, 2, 3, 1, 1, 1, 2, 2,
       3, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 2, 3, 2, 1, 2, 2, 1, 3,
       1, 1])

In [83]:
df_submission_cluster = pd.DataFrame({'serial number': df_orginal_test['serial number'].values})
df_submission_cluster['cluster_number'] = np.array(cluster_result)
df_submission_cluster.to_csv('submission_ensembles_gbms_cluster.csv', index=None)